<a href="https://colab.research.google.com/github/spirosdouk/hy485-/blob/main/485%5B2025%5D_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SVM Classification – Breast Cancer Dataset
In this exercise, you'll explore Support Vector Machines (SVMs) for binary classification using the Breast Cancer Wisconsin dataset. You'll go beyond just training a model — you'll investigate how SVMs make decisions, visualize decision boundaries, compare kernels, tune hyperparameters, and evaluate model confidence.

## Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer

##Load & Explore the Breast Cancer Dataset

Use the built-in breast cancer dataset and get the basic characteristics for the data.

In [ ]:
data = load_breast_cancer()

# Explore basic characteristics of the dataset
print("Dataset Keys:", data.keys())
print("Data Shape:", data.data.shape)              # (n_samples, n_features)
print("Feature Names:", data.feature_names)
print("Target Names:", data.target_names)

# Convert the dataset into a Pandas DataFrame
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Display the first 5 rows
print(df.head())

# Get basic summary statistics
print(df.describe())

**Question:** What kind of classification problem is this? Are the classes balanced?

## Preprocess the Data
Standardize the features and split the data.

## Visualize with PCA (2D)

Use PCA to reduce the feature space to 2D and plot the points.

**Question:** Can the classes be separated linearly in this reduced space?

## Train & Evaluate a Linear SVM

Train an SVM with a linear kernel and evaluate its performance.

## Visualize Decision Boundary (with PCA Projection)

Project the decision surface into PCA space (from previous task).


## Train an SVM with RBF Kernel

Use a radial basis function kernel and compare its performance.

## Cross-Validation Comparison

Compare linear vs. RBF kernel using 5-fold cross-validation.

## Grid Search Hyperparameter Tuning (RBF)
Perform a grid search over `C` and `gamma` for the RBF kernel.

## Evaluate Model Confidence (Decision Function)

Plot the decision function (histogram) or prediction confidence on the PCA plot.


**Question:** Are predictions well-separated or clustered near the margin?